# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.metrics import multilabel_confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn import svm

from sqlalchemy import create_engine

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Felipe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Felipe\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Felipe\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# load data from database
engine = create_engine('sqlite:///clean_f8.db')
df = pd.read_sql_table('clean_f8_2', con=engine)

X = df['message']
Y = df[df.columns[4:]]

### 2. Write a tokenization function to process your text data

In [3]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, 'urlplaceholder')
        
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
        
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
        ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

#pipeline.fit(X_train, Y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
Y_pred = pipeline.predict(X_test)

In [8]:
categories = Y.columns.values
for i, cat in enumerate(categories):
    print(cat)
    print(classification_report(Y_test.iloc[:,i], Y_pred[:,i]))

related
              precision    recall  f1-score   support

           0       0.74      0.26      0.39      1527
           1       0.81      0.97      0.88      5027

    accuracy                           0.81      6554
   macro avg       0.77      0.62      0.64      6554
weighted avg       0.79      0.81      0.77      6554

request
              precision    recall  f1-score   support

           0       0.90      0.99      0.94      5458
           1       0.91      0.43      0.58      1096

    accuracy                           0.90      6554
   macro avg       0.90      0.71      0.76      6554
weighted avg       0.90      0.90      0.88      6554

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6523
           1       0.00      0.00      0.00        31

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      1.00      0.99      655

C:\Users\Felipe\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Felipe\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Felipe\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.94      0.99      0.96      5881
           1       0.85      0.44      0.58       673

    accuracy                           0.93      6554
   macro avg       0.89      0.72      0.77      6554
weighted avg       0.93      0.93      0.93      6554

shelter
              precision    recall  f1-score   support

           0       0.93      1.00      0.96      5974
           1       0.93      0.17      0.29       580

    accuracy                           0.93      6554
   macro avg       0.93      0.59      0.63      6554
weighted avg       0.93      0.93      0.90      6554

clothing
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6459
           1       0.64      0.07      0.13        95

    accuracy                           0.99      6554
   macro avg       0.81      0.54      0.56      6554
weighted avg       0.98      0.99      0.98      6554

mo

C:\Users\Felipe\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Felipe\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Felipe\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Felipe\anaconda3\lib\site-packages\sklearn\

### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__ccp_alpha', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__max_samples', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf_

In [12]:
parameters = {
    'tfidf__smooth_idf': (True, False),
    'clf__estimator__n_estimators': [100, 200],
#    'clf__estimator__': (),
#    'clf__estimator__': (),
}

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=3)

In [6]:
parameters = {
    'clf__estimator':  (RandomForestClassifier(), svm.SVC())
}

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=3)

In [7]:
cv.fit(X_train, Y_train)

Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV 1/5] END clf__estimator=RandomForestClassifier();, score=0.254 total time= 6.0min
[CV 2/5] END clf__estimator=RandomForestClassifier();, score=0.246 total time= 5.9min
[CV 3/5] END clf__estimator=RandomForestClassifier();, score=0.242 total time= 5.9min
[CV 4/5] END clf__estimator=RandomForestClassifier();, score=0.228 total time= 5.8min
[CV 5/5] END clf__estimator=RandomForestClassifier();, score=0.240 total time= 5.8min


C:\Users\Felipe\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Felipe\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Felipe\anaconda3\lib\site-packages\sklearn\pipeline.py", line 346, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\Felipe\anaconda3\lib\site-packages\sklearn\multioutput.py", line 368, in fit
    super().fit(X, Y, sample_weight, **fit_params)
  File "C:\Users\Felipe\anaconda3\lib\site-packages\sklearn\multioutput.py", line 178, in fit
    self.estimators_ = Parallel(n_jobs=self.n_jobs)(
  File "C:\Users\Felipe\anaconda3\lib\site-packages\joblib\parallel.py", line 1046, in __call__
    while self.dispatch_one_batch(it

[CV 1/5] END ................clf__estimator=SVC();, score=nan total time=12.3min


C:\Users\Felipe\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Felipe\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Felipe\anaconda3\lib\site-packages\sklearn\pipeline.py", line 346, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\Felipe\anaconda3\lib\site-packages\sklearn\multioutput.py", line 368, in fit
    super().fit(X, Y, sample_weight, **fit_params)
  File "C:\Users\Felipe\anaconda3\lib\site-packages\sklearn\multioutput.py", line 178, in fit
    self.estimators_ = Parallel(n_jobs=self.n_jobs)(
  File "C:\Users\Felipe\anaconda3\lib\site-packages\joblib\parallel.py", line 1046, in __call__
    while self.dispatch_one_batch(it

[CV 2/5] END ................clf__estimator=SVC();, score=nan total time=12.2min


...

C:\Users\Felipe\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Felipe\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Felipe\anaconda3\lib\site-packages\sklearn\pipeline.py", line 346, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\Felipe\anaconda3\lib\site-packages\sklearn\multioutput.py", line 368, in fit
    super().fit(X, Y, sample_weight, **fit_params)
  File "C:\Users\Felipe\anaconda3\lib\site-packages\sklearn\multioutput.py", line 178, in fit
    self.estimators_ = Parallel(n_jobs=self.n_jobs)(
  File "C:\Users\Felipe\anaconda3\lib\site-packages\joblib\parallel.py", line 1046, in __call__
    while self.dispatch_one_batch(it

[CV 4/5] END ................clf__estimator=SVC();, score=nan total time=12.4min


C:\Users\Felipe\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Felipe\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Felipe\anaconda3\lib\site-packages\sklearn\pipeline.py", line 346, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\Felipe\anaconda3\lib\site-packages\sklearn\multioutput.py", line 368, in fit
    super().fit(X, Y, sample_weight, **fit_params)
  File "C:\Users\Felipe\anaconda3\lib\site-packages\sklearn\multioutput.py", line 178, in fit
    self.estimators_ = Parallel(n_jobs=self.n_jobs)(
  File "C:\Users\Felipe\anaconda3\lib\site-packages\joblib\parallel.py", line 1046, in __call__
    while self.dispatch_one_batch(it

[CV 5/5] END ................clf__estimator=SVC();, score=nan total time=12.4min


GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x000001BA9F3F8CA0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator': (RandomForestClassifier(), SVC())},
             verbose=3)

In [8]:
Y_pred = cv.predict(X_test)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [9]:
    labels = np.unique(Y_pred)
    confusion_mat = multilabel_confusion_matrix(Y_test, Y_pred, labels=labels)
    accuracy = (Y_pred == Y_test).mean()

    print("Labels:", labels)
    print("Confusion Matrix:\n", confusion_mat)
    print("Model Accuracy:", accuracy.mean())
    print("Accuracy Breakdown:", accuracy)
    print("\nBest Parameters:", cv.best_params_)

Labels: [0 1]
Confusion Matrix:
 [[[ 412 1111]
  [ 132 4899]]

 [[5376   76]
  [ 617  485]]]
Accuracy: related                   0.810345
request                   0.894263
offer                     0.996186
aid_related               0.772200
medical_help              0.926762
medical_products          0.951938
search_and_rescue         0.971926
security                  0.981843
military                  0.970247
child_alone               1.000000
water                     0.950565
food                      0.926457
shelter                   0.927525
clothing                  0.987336
money                     0.979249
missing_people            0.987031
refugees                  0.965060
death                     0.957431
other_aid                 0.873970
infrastructure_related    0.931187
transport                 0.957583
buildings                 0.949496
electricity               0.980623
tools                     0.993592
hospitals                 0.988557
shops                 

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [13]:
categories = Y.columns.values
for i, cat in enumerate(categories):
    print(cat)
    print(classification_report(Y_test.iloc[:,i], Y_pred[:,i]))


related
              precision    recall  f1-score   support

           0       0.76      0.27      0.40      1523
           1       0.82      0.97      0.89      5031

    accuracy                           0.81      6554
   macro avg       0.79      0.62      0.64      6554
weighted avg       0.80      0.81      0.77      6554

request
              precision    recall  f1-score   support

           0       0.90      0.99      0.94      5452
           1       0.86      0.44      0.58      1102

    accuracy                           0.89      6554
   macro avg       0.88      0.71      0.76      6554
weighted avg       0.89      0.89      0.88      6554

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6529
           1       0.00      0.00      0.00        25

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      1.00      0.99      655

C:\Users\Felipe\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Felipe\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Felipe\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.93      0.99      0.96      5838
           1       0.85      0.40      0.54       716

    accuracy                           0.93      6554
   macro avg       0.89      0.69      0.75      6554
weighted avg       0.92      0.93      0.91      6554

shelter
              precision    recall  f1-score   support

           0       0.93      0.99      0.96      5973
           1       0.82      0.23      0.36       581

    accuracy                           0.93      6554
   macro avg       0.87      0.61      0.66      6554
weighted avg       0.92      0.93      0.91      6554

clothing
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6463
           1       1.00      0.09      0.16        91

    accuracy                           0.99      6554
   macro avg       0.99      0.54      0.58      6554
weighted avg       0.99      0.99      0.98      6554

mo

C:\Users\Felipe\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Felipe\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Felipe\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Felipe\anaconda3\lib\site-packages\sklearn\

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6512
           1       0.00      0.00      0.00        42

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      0.99      0.99      6554

hospitals
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6479
           1       0.00      0.00      0.00        75

    accuracy                           0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.98      0.99      0.98      6554

shops
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6515
           1       0.00      0.00      0.00        39

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      0.99      0.99      6554

aid

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.